In [ ]:
import json
import pandas as pd
import glob
from collections import defaultdict
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from tokenizers import Tokenizer
from transformers import pipeline
import warnings
import os

## Ko-Bart 

사투리 -> 표준어 번역 프로세싱

In [ ]:
dir = "/kaggle/input/jeju-labeled-trainset"

files = glob.glob(f"{dir}/*.json")
df = pd.DataFrame(index=range(2), columns=["사투리","표준어"])

for file in files:
    _dict = defaultdict(list)
    
    with open(file) as f:
        json_data = json.load(f)
        # JSON 파일의 utterance 키 값 아래에 리스트형태로 데이터가 존재했다.
        data = json_data.get("utterance")
        
        for item in data:
            # standard_form(표준어)와 dialect_form(사투리)가 일치하지 않는다면?
            standard = item.get("standard_form")
            dialect = item.get("dialect_form")
            if standard != dialect:
                _dict["표준어"].append(standard)
                _dict["사투리"].append(dialect)
    
    # 한 JSON 파일에서 데이터 추출이 끝나면 비어있던 데이터프레임에 추가
    append_df = pd.DataFrame(_dict)
    df = (append_df)

df.to_csv("data.tsv",sep="\t")

In [ ]:
df.tail()

In [ ]:
import re

step1 = re.compile(r'&\w+&')
step2 = re.compile(r'\(*\)')
step3 = re.compile(r'\{[^}]*\}')
step4 = re.compile(r'\-[^}]*\-')

#예시 문장
text1 = "&name2& 님은 뭐~ {laughing} (()) 힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?"
text2 = "어~ {반가움} &name123&! (()) 반갑고 ㅋㅋ -오- 오랜만이다"

# 무식해보이지만 일단 확인해보자!
for text in [text1,text2]:
    after_step1=step1.sub('',text)
    after_step2=step2.sub('',after_step1)
    after_step3=step3.sub('',after_step2)
    after_step4=step4.sub('',after_step3)
    
    print(f"Text : {text}")
    print(f"Step 1 : {after_step1}")
    print(f"Step 2 : {after_step2}")
    print(f"Step 3 : {after_step3}")
    print(f"Step 4 : {after_step4}")
    print("\n\n")

## 데이터 정제

In [ ]:
df = pd.read_csv(f'/kaggle/working/data.tsv',sep='\t')

# data.tsv를 불러왔더니 nan 값이 들어있어 일단 삭제해줬다.
df = df.dropna()

# 아까 지정해뒀던 정규표현식들
step1 = re.compile(r'&\w+&')
step2 = re.compile(r'\(*\)')
step3 = re.compile(r'\{[^}]*\}')
step4 = re.compile(r'\-[^}]*\-')


def make_cleaned_text(text):
    after_step1=step1.sub('',text)
    after_step2=step2.sub('',after_step1)
    after_step3=step3.sub('',after_step2)
    after_step4=step4.sub('',after_step3)

    return after_step4

for idx, row in df.iterrows():
    df.at[idx, '사투리'] = make_cleaned_text(row['사투리'])
    df.at[idx, '표준어'] = make_cleaned_text(row['표준어'])

df.to_csv('/kaggle/working/data_fixed.tsv',sep='\t')

In [ ]:
df.tail()

## 모델 파라미터 설정

In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

# 모델 학습 후에 가중치 저장할 폴더
model_path='/kaggle/working/saved_model'
# 모델 초기 가중치 로드할 곳
model_name = "gogamza/kobart-base-v2"
# 데이터셋 파일
data_root='/kaggle/working'

# 내가 학습한 모델 가중치의 유무에 따라 분기 처리
if os.path.exists(f'{model_path}/pytorch_model.bin'):
    print("Use Customized Model")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
else:
    print("Use Pretrained Model")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Training Arguments

args = {
    'num_train_epochs': 10,
    'per_device_train_batch_size': 32,
    'per_device_eval_batch_size': 32,
    'overwrite_output_dir': True,
    'eval_steps': 3000,
    'save_steps': 6000,
    'warmup_steps': 300,
    'evaluation_strategy': "steps",
    'prediction_loss_only': True,
    'save_total_limit': 3
}

In [ ]:
class TextStyleTransferDataset(Dataset):
    def __init__(self, df,tokenizer):
        self.df = df
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row=self.df.iloc[index]
        text1=row[0]
        text2=row[1]
        target_style_name = '표준어'
		
        # Tokenizer를 허깅페이스의 레포지토리에서 가져왔기 때문에
        # 테스트 코드를 참고해서 모델 입력 형식을 만들었다.
        encoder_text = f"{target_style_name} 말투로 변환:{text1}"
        decoder_text = f"{text2}{self.tokenizer.eos_token}"
        model_inputs = self.tokenizer(encoder_text, max_length=64, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            labels = tokenizer(decoder_text, max_length=64, truncation=True)
        model_inputs['labels'] = labels['input_ids']
        del model_inputs['token_type_ids']

        return model_inputs

    # 지난번에 만들어뒀던 데이터셋을 불러와서 데이터프레임을 만들어준다
    def make_df(data_root):
        df = pd.read_csv(f'{data_root}/data.tsv',sep='\t')
        # 주로 쓰이는 방법 같지는 않지만,, Train Data와 Test Data를 8:2 비율로 나눠준다
        rate=int(len(df)*0.2)
        df_train,df_test = df[rate:],df[:rate]

        print(f'Train DataFrame length : {len(df_train)},Test DataFrame length : {len(df_test)}')
        return df_train,df_test
    
    def make_dataset(data_root,tokenizer):
    df_train,df_test = make_df(data_root)

    train_dataset = TextStyleTransferDataset(df_train,tokenizer)
    test_dataset = TextStyleTransferDataset(df_test,tokenizer)

        return train_dataset,test_dataset

## 학습

In [ ]:
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer,\
                         DataCollatorForSeq2Seq
warnings.filterwarnings("ignore")


train_dataset,test_dataset = TextStyleTransferDataset.make_dataset(
    data_root, tokenizer
)


data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)


training_args = Seq2SeqTrainingArguments(
    **args,
    output_dir=model_path,
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


In [ ]:
# 모델 학습 진행
try:
    trainer.train()
except Exception as e:
    print(f"Failed to train model caused by {e}")
  

In [ ]:
try:
    trainer.save_model(model_path)
    print("Model saved successfully.")
except Exception as e:
    print(f"Failed to save model caused by {e}")

## Test (Validation)

AI hub에 있는 텍스트 데이터에는 테스트용(라벨 없는거)이 따로 없긴 하다.
그래서 이전에 그냥 trainset에서 8:2로 split시켜서 validation용으로 퉁치긴 했지만
일단 나중에 오디오 -> 텍스트(STT)실행하고 나서 어차피 데이터는 바꿔서 껴야되기 때문에 그때 validation도 상황에 맞게 데이터 변경해야함


In [ ]:
pip install deep-translater

In [ ]:
from transformers import pipeline
from deep_translator import GoogleTranslator

nlg_pipeline=pipeline('text2text-generation',model=config.model_path,tokenizer=config.model_name)

In [ ]:
def generate_text(pipe, text, num_return_sequences, max_length):
    target_style_name = "표준어"
    text = f"{target_style_name} 말투로 변환:{text}"
    out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
    return [x['generated_text'] for x in out]